In [5]:
import pandas as pd
import numpy as np

# 一、Groupby

In [6]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.069724,-0.001161,a,one
1,-1.498952,0.609066,a,two
2,2.146232,0.879019,b,one
3,-0.210835,0.379417,b,two
4,0.049598,-0.419212,a,one


##  1. 按列分组：

### 注意以下使用groupby()函数生成的group1是一个中间分组变量，为GroupBy类型。

In [7]:
group1 = df.groupby('key1')
group1

In [8]:
group2 = df.groupby(['key1','key2'])
group2

## 2. 使用推导式[x for x in group1]可显示分组内容。

In [9]:
[x for x in group1]

[('a',       data1     data2 key1 key2
  0 -0.069724 -0.001161    a  one
  1 -1.498952  0.609066    a  two
  4  0.049598 -0.419212    a  one), ('b',       data1     data2 key1 key2
  2  2.146232  0.879019    b  one
  3 -0.210835  0.379417    b  two)]

## 3. 按分组统计
### 在分组group1、group2上应用size()、sum()、count()等统计函数，能分别统计分组数量、不同列的分组和、不同列的分组数量。

In [10]:
group1.size()

key1
a    3
b    2
dtype: int64

In [11]:
group1.sum()

,data1,data2
key1,,
a,-1.519077,0.188693
b,1.935397,1.258436


In [12]:
group2.size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [13]:
group2.count()

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       1      1
     two       1      1

##  4. 应用agg()：

### 对于分组的某一列或者多个列，应用agg(func)可以对分组后的数据应用func函数。

### 例如：用group1['data1'].agg('mean')对分组后的’data1’列求均值。当然也可以推广到同时作用于多个列和使用多个函数上。

In [14]:
group1['data1'].agg('mean')

key1
a   -0.506359
b    0.967698
Name: data1, dtype: float64

In [15]:
group1['data1'].agg(['mean','sum'])

,mean,sum
key1,,
a,-0.506359,-1.519077
b,0.967698,1.935397


In [16]:
group1['data1','data2'].agg(['mean','sum'])

data1               data2          
          mean       sum      mean       sum
key1                                        
a    -0.506359 -1.519077  0.062898  0.188693
b     0.967698  1.935397  0.629218  1.258436

## 5. 应用apply()

### apply()不同于agg()的地方在于：前者应用于dataframe的各个列，后者仅作用于指定的列。

In [20]:
df.groupby('key1').apply(np.mean)

,data1,data2
key1,,
a,-0.506359,0.062898
b,0.967698,0.629218


In [21]:
df.groupby(['key1','key2']).apply(np.mean)

data1     data2
key1 key2                    
a    one  -0.010063 -0.210187
     two  -1.498952  0.609066
b    one   2.146232  0.879019
     two  -0.210835  0.379417

# 二、透视表pivot_table 

### 可以产生类似于excel数据透视表的结果，相当的直观。

## 1. 分组统计

### 其中参数index指定“行”键，columns指定“列”键。

In [22]:
df

,data1,data2,key1,key2
0,-0.069724,-0.001161,a,one
1,-1.498952,0.609066,a,two
2,2.146232,0.879019,b,one
3,-0.210835,0.379417,b,two
4,0.049598,-0.419212,a,one


In [23]:
pd.pivot_table(df, index='key1', columns='key2')

data1               data2          
key2       one       two       one       two
key1                                        
a    -0.010063 -1.498952 -0.210187  0.609066
b     2.146232 -0.210835  0.879019  0.379417

In [30]:
df.pivot_table(['data1'], index='key1',columns='key2', aggfunc=np.sum)

data1          
key2       one       two
key1                    
a    -0.020125 -1.498952
b     2.146232 -0.210835

## 2. 分项汇总

### 如果将参数margins设置为True，则可以得到分项总计数据。

In [31]:
df.pivot_table(index='key1',columns='key2', margins=True)

data1                         data2                    
key2       one       two       All       one       two       All
key1                                                            
a    -0.010063 -1.498952 -0.506359 -0.210187  0.609066  0.062898
b     2.146232 -0.210835  0.967698  0.879019  0.379417  0.629218
All   0.708702 -0.854894  0.083264  0.152882  0.494241  0.289426

# 三、交叉表crosstab 

### 可以按照指定的行和列统计分组频数，用起来非常方便；当然同样的功能也可采用groupby实现。

In [32]:
pd.crosstab(df.key1,df.key2, margins=True)

key2,one,two,All
key1,,,
a,2,1,3
b,1,1,2
All,3,2,5
